In [1]:

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
import seaborn as sns

Mounted at /content/drive


In [67]:
df = pd.read_csv('/content/drive/My Drive/kaggle desafios/clothes/final_test.csv')

In [ ]:
df.head(10)

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
sns.pairplot(data=df, hue='size', height=7)
plt.show()

## Removendo outliers usando zscore, de acordo com https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

In [ ]:
dfs = []
sizes = []
for size_type in df['size'].unique():
    print('size type:',size_type)
    sizes.append(size_type)
    ndf = df[['age','height','weight']][df['size'] == size_type]
    zscore = ((ndf - ndf.mean())/ndf.std())
    #print(zscore)
    dfs.append(zscore)

In [ ]:
for i in range(len(dfs)):
    print(sizes[i])
    dfs[i]['age'] = dfs[i]['age'][(dfs[i]['age']>-3) & (dfs[i]['age']<3)]
    dfs[i]['height'] = dfs[i]['height'][(dfs[i]['height']>-3) & (dfs[i]['height']<3)]
    dfs[i]['weight'] = dfs[i]['weight'][(dfs[i]['weight']>-3) & (dfs[i]['weight']<3)]
    dfs[i]['size'] = sizes[i]

Eu decidi deletar valores NaN porque a quantidade não é muito grande em relação ao total dos dados, mas uma opção seria preencher com a media com base no "size" que esta classificado o valor faltante

In [109]:
df = pd.concat(dfs)

In [110]:
df.dropna(inplace = True)

In [75]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier

In [76]:
from sklearn.tree import DecisionTreeClassifier

In [77]:
from sklearn.model_selection import train_test_split

transformando os tamanhos em dados numericos e criando novas features

In [111]:
labelEncoder = LabelEncoder()

In [112]:
size_enconder = labelEncoder.fit_transform(df['size'])

In [113]:
df['size_enc'] = size_enconder

In [114]:
df['imc'] = df['weight'] / (df['height']/100 * df['height']/100)

In [115]:
df['weight2'] = df['weight'] * df['weight']

separando o df de treino

In [116]:
df_train = df.copy()

In [ ]:
df_train.isna().sum()

In [118]:
df_train.drop(columns= ['size'], inplace = True, axis = 1) 

In [119]:
df_labels = df_train['size_enc']
df_train.drop(columns = ['size_enc'], inplace = True, axis = 1)

In [120]:
x_train, x_test, y_train, y_test = train_test_split(df_train, df_labels, test_size=0.2, random_state=0)

In [ ]:
rede_neural = MLPClassifier(max_iter = 1000, 
                            verbose = True,
                            tol = 0.00001,
                            solver = 'adam',
                            activation = 'relu',
                            hidden_layer_sizes = (6,6))

In [122]:
from sklearn.metrics import accuracy_score, classification_report

In [131]:
rede_neural = MLPClassifier(max_iter = 1000, 
                        tol = 0.00001,
                        solver = 'adam',
                        activation = 'tanh',
                        hidden_layer_sizes = (20,20))

In [132]:
rede_neural.fit(x_train, y_train)
prev = rede_neural.predict(x_test)
print(accuracy_score(y_test, prev))

0.2649976381672178


In [ ]:
for s in ['adam', 'sgd']:
  for act in ['identity', 'logistic', 'tanh', 'relu']:
    
    print('\n ----------------\n ',s)
    print(act)
    rede_neural = MLPClassifier(max_iter = 1000, 
                            tol = 0.00001,
                            solver = s,
                            activation = act,
                            hidden_layer_sizes = (5,5))
    rede_neural.fit(x_train, y_train)
    prev = rede_neural.predict(x_test)
    print(accuracy_score(y_test, prev))
    #print(classification_report(y_test, prev))

#Random Forest

In [125]:
from sklearn.ensemble import RandomForestClassifier

In [126]:
for n in range(20, 300, 20):
  RFC = RandomForestClassifier(n_estimators= n, criterion = 'entropy')
  RFC.fit(x_train, y_train)
  prev = RFC.predict(x_test)
  ac = accuracy_score(y_test, prev)
  print(n, ac)

20 0.999269978958217
40 0.9993129213724395
60 0.9992270365439945
80 0.9993129213724395
100 0.999269978958217
120 0.999269978958217
140 0.9991840941297719
160 0.999269978958217
180 0.9991840941297719
200 0.9991840941297719
220 0.9993129213724395
240 0.9992270365439945
260 0.9992270365439945
280 0.9991840941297719
